# Clustering Crypto

In [46]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [47]:
# Load the crypto_data.csv dataset.
file_path = Path("./Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head(10)
crypto_df.shape

(1252, 6)

In [48]:
# Keep all the cryptocurrencies that are being traded.
crypto_trading_df = crypto_df[crypto_df['IsTrading'] == True]
print(crypto_trading_df.shape)
crypto_trading_df.head(10)

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [49]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_working_alg_df = crypto_trading_df[crypto_trading_df['Algorithm'] != 'NaN']
print(crypto_working_alg_df.shape)
crypto_working_alg_df.head(10)

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [50]:
# Remove the "IsTrading" column. 
crypto_isTrading_drop_df = crypto_working_alg_df.drop(columns=['IsTrading']).copy()
print(crypto_isTrading_drop_df.shape)
crypto_isTrading_drop_df.head(10)

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [51]:
# Remove rows that have at least 1 null value.
crypto_isTrading_drop_df.isnull().sum()
crypto_mineless_coins_df = crypto_isTrading_drop_df.dropna()
print(crypto_mineless_coins_df.shape)
crypto_mineless_coins_df.head(10)

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [52]:
# Keep the rows where coins are mined.
crypto_mineless_coins_df = crypto_mineless_coins_df[crypto_mineless_coins_df['TotalCoinsMined'] != 0].copy()
print(crypto_mineless_coins_df.shape)
crypto_mineless_coins_df.head(10)

(533, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [53]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name_df = crypto_mineless_coins_df[['CoinName']]
print(coin_name_df.shape)
coin_name_df.head(10)

(533, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
LTC,Litecoin
DASH,Dash
XMR,Monero
ETC,Ethereum Classic
ZEC,ZCash


In [54]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
coinName_drop_df = crypto_mineless_coins_df.drop(columns=['CoinName']).copy()
print(coinName_drop_df.shape)
coinName_drop_df.head(10)

(533, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [55]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(coinName_drop_df, columns=['Algorithm', 'ProofType'])
print(X.shape)
X.head(10)

(533, 100)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
# Standardize the data with StandardScaler().
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# print(X_scaled[0:5])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [57]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3, random_state=1)
crypto_pca = pca.fit_transform(X_scaled)

In [58]:
# Create a DataFrame with the three principal components.
X_index = X.index

pcs_df = pd.DataFrame(
    data = crypto_pca, columns=["PC 1", "PC 2", "PC 3"] 
).set_index(X_index)
print(pcs_df.shape)
pcs_df.head(10)


(533, 3)


,PC 1,PC 2,PC 3
42,-0.336211,0.991851,-0.611739
404,-0.319516,0.992137,-0.611933
1337,2.309454,1.693327,-0.630987
BTC,-0.141254,-1.276005,0.173388
ETH,-0.145039,-1.973940,0.387062
LTC,-0.165299,-1.156925,-0.028976
DASH,-0.390365,1.243133,-0.525666
XMR,-0.140180,-2.227158,0.368806
ETC,-0.143477,-1.974029,0.387072
ZEC,-0.147357,-2.035769,0.365626


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [59]:
# Create an elbow curve to find the best value for K.
inertia =[]
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=1)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [60]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [61]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([coinName_drop_df, pcs_df], axis=1)
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df, coin_name_df], axis=1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.336211,0.991851,-0.611739,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.319516,0.992137,-0.611933,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.309454,1.693327,-0.630987,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.141254,-1.276005,0.173388,Bitcoin,2
ETH,Ethash,PoW,1.076842e+08,0,-0.145039,-1.973940,0.387062,Ethereum,2
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.165299,-1.156925,-0.028976,Litecoin,2
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.390365,1.243133,-0.525666,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.140180,-2.227158,0.368806,Monero,2
ETC,Ethash,PoW,1.133597e+08,210000000,-0.143477,-1.974029,0.387072,Ethereum Classic,2
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.147357,-2.035769,0.365626,ZCash,2


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [62]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE


In [63]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [64]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [65]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [66]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

NameError: name 'plot_df' is not defined

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
